In [233]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
import cv2
import numpy as np 
import os 
import pandas as pd
import matplotlib.image as mpimg
from PIL import Image
import keras_tuner as kt


In [234]:
def load_images_labels(image_folder, label_file, target_size=(64,64)):
    images, labels = [], []
    
    label_df = pd.read_csv(label_file)
    label_dict = dict(zip(label_df['IMAGE'], label_df['MEDICINE_NAME'])) 
    
    total_files = len(os.listdir(image_folder))
    loaded_files = 0
    skipped_files = 0
    
    for img_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_file)
        image = Image.open(img_path).convert('RGB')  
        image = image.resize(target_size)  
        image_array = np.array(image)  
        # print(image_array.shape)
        images.append(image_array)
        labels.append(label_dict[img_file])

    return np.array(images), np.array(labels)

In [235]:
train_img_folder = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Training/training_words"
train_label_file = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv"

test_img_folder = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words"
test_label_file = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Testing/testing_labels.csv"

val_img_folder = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Validation/validation_words"
val_label_f = "archive (2)/Doctor’s Handwritten Prescription BD dataset/Validation/validation_labels.csv"


In [236]:
im = mpimg.imread("archive (2)/Doctor’s Handwritten Prescription BD dataset/Training/training_words/8.png")

print(im.shape)


(66, 177, 3)


In [237]:
X_train, y_train = load_images_labels(train_img_folder, train_label_file)
print("-------------------------------------------------------")
X_val, y_val = load_images_labels(val_img_folder, val_label_file)
print("-------------------------------------------------------")
X_test, y_test = load_images_labels(test_img_folder, test_label_file)

-------------------------------------------------------
-------------------------------------------------------


In [238]:
len(X_train)
X_train.shape

(3120, 64, 64, 3)

In [239]:

X_train, X_val, X_test = np.array(X_train, dtype='float32') / 255.0, np.array(X_val, dtype='float32') / 255.0, np.array(X_test, dtype='float32') / 255.0

print("Initial shapes:", X_train.shape, X_val.shape, X_test.shape)

if X_train.ndim == 3:  
    X_train = np.expand_dims(X_train, axis=-1)
if X_val.ndim == 3:
    X_val = np.expand_dims(X_val, axis=-1)
if X_test.ndim == 3:
    X_test = np.expand_dims(X_test, axis=-1)

print("Shapes after expansion:", X_train.shape, X_val.shape, X_test.shape)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

y_train, y_val, y_test = np.array(y_train, dtype='int32'), np.array(y_val, dtype='int32'), np.array(y_test, dtype='int32')





Initial shapes: (3120, 64, 64, 3) (780, 64, 64, 3) (780, 64, 64, 3)
Shapes after expansion: (3120, 64, 64, 3) (780, 64, 64, 3) (780, 64, 64, 3)


In [246]:
# Model definition with Keras Tuner
IMG_SIZE = (64,64,3)
def build_model(hp):
    model = Sequential()
    
    # Tuning Conv2D layers
    for i in range(hp.Int('conv_layers', 2, 4)): 
        model.add(Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]),
            activation='relu',
            input_shape=(IMG_SIZE[0], IMG_SIZE[1], IMG_SIZE[2]) if i == 0 else None
        ))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Flatten and Dense layers
    model.add(Flatten())
    model.add(Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dense_dropout', min_value=0.3, max_value=0.6, step=0.1)))
    
    # Output layer
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [248]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  
    executions_per_trial=1, 
    directory='kt_search_dir',
    project_name='handwritten_prescription_tuning'
)

Reloading Tuner from kt_search_dir\handwritten_prescription_tuning\tuner0.json


In [249]:
tuner.search(X_train, y_train, batch_size=32,
             validation_data=(X_val, y_val),
             epochs=10)


Trial 10 Complete [00h 06m 35s]
val_accuracy: 0.656410276889801

Best val_accuracy So Far: 0.7384615540504456
Total elapsed time: 01h 27m 29s


In [ ]:
# as you can see on 10 trials of  random hyperparameters generated  by keras tuner for cnn model
# the max accuracy coming as 73.8 %.
# on net i saw a file where one person is getting 74 % accuracy but he fit and eavluate the model on same dataset not on seperate 
# like validation dataset and test dataset 